In [ ]:
# import requests
#https://ca.global.nba.com/statistics/
page = requests.get("https://ca.global.nba.com/statistics/")

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
#

blocs = set()
for bloc in soup.find_all('div', class_='sib-table-full-player-stats'):
        print(bloc)


In [25]:
#Import de la librairie pandas
import pandas

#Lecture des fichiers de données en spécifiant le séparateur point-virgule ';'
equipeJO = pandas.read_csv("20162020_JO_Team.csv", sep=';')

joueurJO2016 = pandas.read_csv("2016_JO_Player.csv", sep=';')
joueurNBA2016 = pandas.read_csv("2016_NBA_Player.csv", sep=';')

#Liste provenant de WashingtonPost. Information peut-être redondante avec le non-WP. 
joueurNBAWP2016 = pandas.read_csv("2016_NBA_Player_WP.csv", sep=';')
equipeNBA2016 = pandas.read_csv("2016_NBA_Team.csv", sep=';')

#2020
joueurJO2020 = pandas.read_csv("2020_JO_Player.csv", sep=';')
joueurNBA2020 = pandas.read_csv("2020_NBA_Player.csv", sep=';')
equipeNBA2020 = pandas.read_csv("2020_NBA_Team.csv", sep=';')

#print(notes.dtypes)
joueurNBA2016

,Rank,Name,Age,Position,Team,Games Played,Field Goals Made Per Game,Field Goals Attempted Per Game,Field Goals Made Percentage,3 Point Made Per Game,3 Point Attempted Per Game,3 Point Made Percentage,Free Throws Made Per Game,Free Throws Attempted Per Game,Free Throws Made Percentage,Rebounds Per Game,Assists per game,Steals Per Game,Blocks per Game,Points Per game
0,1,Russell Westbrook,27,Point Guard,Oklahoma City Thunder,81,10.2,24.0,42.5,2.5,7.2,34.3,8.8,10.4,84.5,10.7,10.4,1.6,0.4,31.6
1,2,James Harden,27,Shooting Guard,Houston Rocks,81,8.3,18.9,44.0,3.2,9.3,34.7,9.2,10.9,84.7,8.1,11.2,1.5,0.5,29.1
2,3,Isaiah Thomas,33,Point Guard,Boston Celtics,76,9.0,19.4,46.3,3.2,8.5,37.9,7.8,8.5,90.9,2.7,5.9,0.9,0.2,28.9
3,4,Anthony Davis,23,Power Forward,New Orleans Pelicans,75,10.3,20.3,50.5,0.5,1.8,29.9,6.9,8.6,80.2,11.8,2.1,1.3,2.2,28.0
4,5,DeMar DeRozan,32,Guard / Forward,Toronto Raptors,74,9.7,20.9,46.7,0.4,1.7,26.6,7.4,8.7,84.2,5.2,3.9,1.1,0.2,27.3
5,6,Damian Lillard,26,Point Guard,Portland Trail Blazers,75,8.8,19.8,44.4,2.9,7.7,37.0,6.5,7.3,89.5,4.9,5.9,0.9,0.3,27.0
6,7,DeMarcus Cousins,26,Center,Sacramento Kings,72,9.0,19.9,45.2,1.8,5.0,36.1,7.2,9.3,77.2,11.0,4.6,1.4,1.3,27.0
7,8,LeBron James,31,Small Forward,Cleveland Cavaliers,74,9.9,18.2,54.8,1.7,4.6,36.3,4.8,7.2,67.4,8.6,8.7,1.2,0.6,26.4
8,9,Kawhi Leonard,25,Small Forward,San Antonio Spurs,74,8.6,17.7,48.5,2.0,5.2,38.0,6.3,7.2,88.0,5.8,3.5,1.8,0.7,25.5
9,10,Stephen Curry,28,Point Guard,Golden State Warriors,79,8.5,18.3,46.8,4.1,10.0,41.1,4.1,4.6,89.8,4.5,6.6,1.8,0.2,25.3


Quel est la corrélation entre la participation aux Olympiques et le classements des joueurs dans la NBA?

In [28]:
# jointure
#df_sales_full = df_sales.join(df_product.set_index("product_id"), on="product_id")

#print(joueurJO2016.columns)
joueur2016 = joueurJO2016.join(joueurNBA2016.where(joueurNBA2016['Rank'] <= 12).set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner').sort_values(by=['Rank'])
joueur2020 = joueurJO2020.join(joueurNBA2020.where(joueurNBA2020['Rank'] <= 12).set_index('Name'), on="Name", lsuffix='JO', rsuffix='NBA', how='inner').sort_values(by=['Rank'])

#joueur2020.transpose()
joueur2016

,Name,AgeJO,TeamJO,PositionJO,Games PlayedJO,Game Started,Field Goals Made,Field Goals Attempted,Field Goals Made PercentageJO,3 Point Made,...,3 Point Attempted Per Game,3 Point Made Percentage,Free Throws Made Per Game,Free Throws Attempted Per Game,Free Throws Made PercentageNBA,Rebounds Per Game,Assists per game,Steals Per Game,Blocks per Game,Points Per game
4,DeMar DeRozan,26.0,NaN,G,7,0,16,27,0.593,0,...,1.7,26.6,7.4,8.7,84.2,5.2,3.9,1.1,0.2,27.3
3,DeMarcus Cousins,25.0,NaN,C,8,5,27,44,0.614,0,...,5.0,36.1,7.2,9.3,77.2,11.0,4.6,1.4,1.3,27.0
8,Kyrie Irving,NaN,NaN,G,8,8,31,65,0.477,9,...,6.1,40.1,4.1,4.6,90.5,3.2,5.8,1.2,0.3,25.2


Est-ce que les joueurs de la NBA ont une performance similaire moyenne en saison régulière comparé aux olympiques (Points, rebound, assist et block)?

Est-ce que la performance moyenne d'une équipe de la NBA est semblable à une équipe Olympique?

Est-ce que l'écart-type est similaire entre les joueurs d'une équipe de la NBA et ceux d'une équipe Olympique?